## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

Importing library

In [0]:
#Basic imports for Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Imports for Delta Lake
from delta.tables import *
import delta

# MLflow for managing the machine learning model
import mlflow
import mlflow.pyfunc

# Hugging Face Transformers for sentiment analysis
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Import for data visualization (optional, for EDA)
import matplotlib.pyplot as plt
import seaborn as sns

# Import for handling dates and times
from datetime import datetime

# Utilities
import os
import torch

Setting partitions

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
from pyspark.sql import SparkSession

# Start a Spark Session
spark = SparkSession.builder \
    .appName("Read S3 Data") \
    .getOrCreate()
    
# Define the path to your S3 bucket
s3_path = "s3a://voc-75-databricks-data/voc_volume/"

# Read the list of files in the directory
file_list_df = spark.read.format("binaryFile").load(s3_path)

In [0]:
# Count the number of files
file_count = file_list_df.count()
print("Number of files in the directory:", file_count)


In [0]:
from pyspark.sql.functions import sum as spark_sum

# Assuming you've already loaded file_list_df with binaryFile format which includes file path and length
file_sizes = spark.read.format("binaryFile") \
    .option("pathGlobFilter", "*.json") \
    .load("s3a://voc-75-databricks-data/voc_volume/") \
    .select("length")

# Calculate total size in bytes
total_size_bytes = file_sizes.agg(spark_sum("length")).collect()[0][0]

# Convert bytes to more readable units, e.g., GB
total_size_gb = total_size_bytes / (1024 ** 3)
print(f"Total data size: {total_size_gb:.2f} GB")


In [0]:
# Get the path of the first file
first_file_path = file_list_df.select("path").first()[0]

# Read the contents of the first file
file_contents = spark.read.text(first_file_path).collect()

# Print the contents of the file
for line in file_contents:
    print(line.value)


## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

# Define the schema based on the provided data dictionary
schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("source_file", StringType(), True)
])

In [0]:
# Assuming the path variable TWEET_SOURCE_PATH is set up in your environment
tweet_source_path = "s3a://voc-75-databricks-data/voc_volume/"

# Setup the read stream
raw_stream = spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "json") \
    .schema(schema) \
    .load(tweet_source_path)


In [0]:
from pyspark.sql.functions import current_timestamp

# Transform the raw data to bronze format
bronze_data = raw_stream.select(
    col("date").alias("date"),
    col("user").alias("user"),
    col("text").alias("text"),
    col("sentiment").alias("sentiment"),
    input_file_name().alias("source_file"),
    current_timestamp().alias("processing_time")
)

In [0]:
# Paths from the environment variables
bronze_checkpoint_path = "/tmp/labuser104917-3013268/bronze.checkpoint"
bronze_delta_path = "/tmp/labuser104917-3013268/bronze.delta"

# Setup the write stream
bronze_stream = bronze_data.writeStream \
    .format("delta") \
    .option("checkpointLocation", bronze_checkpoint_path) \
    .option("path", bronze_delta_path) \
    .outputMode("append") \
    .queryName("bronze_stream") \
    .start()


In [0]:
print(bronze_stream.status)

In [0]:
BRONZE_DELTA = "/tmp/labuser104917-3013268/bronze.delta"

# Function to print the head of a Delta table
def print_table_head(path_name, path, n=5):
    try:
        df = spark.read.format("delta").load(path)
        count = df.count()
        if count == 0:
            print(f"{path_name} is empty.")
        else:
            print(f"{path_name} is not empty. Row count: {count}")
            df.show(n)
    except Exception as e:
        print(f"Error reading Delta table {path_name} at {path}: {e}")

# Print the head of the Bronze table
print_table_head("Bronze Delta", BRONZE_DELTA, n=5)


In [0]:
# Allow new schema to be merged into the existing Delta table
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")


In [0]:
bronze_stream.awaitTermination()

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
bronze_delta_path = "/tmp/labuser104917-3013268/bronze.delta"
bronze_df = spark.read.format("delta").load(bronze_delta_path)
print(bronze_df.head())

In [0]:
total_tweets = bronze_df.count()
print(f"Total number of tweets captured in the Bronze Table: {total_tweets}")


In [0]:
from pyspark.sql.functions import col, count, when

# Check for null values in each column
nan_null_stats = bronze_df.select([count(when(col(c).isNull(), c)).alias(c) for c in bronze_df.columns])
nan_null_stats.show()


In [0]:
from pyspark.sql.functions import expr

tweets_with_mention = bronze_df.filter(expr("text LIKE '%@%'")).count()
tweets_without_mention = total_tweets - tweets_with_mention

print(f"Number of tweets with at least one mention: {tweets_with_mention}")
print(f"Number of tweets with no mentions: {tweets_without_mention}")


In [0]:
from pyspark.sql.functions import desc

# Count tweets per user and order by count in descending order
user_tweet_counts = bronze_df.groupBy("user").count().orderBy(desc("count"))
top_tweeters = user_tweet_counts.limit(20)
top_tweeters_pd = top_tweeters.toPandas()

In [0]:
print(top_tweeters_pd)

In [0]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
plt.bar(top_tweeters_pd['user'], top_tweeters_pd['count'], color='blue')
plt.xlabel('User Handles')
plt.ylabel('Number of Tweets')
plt.title('Top 20 Tweeters')
plt.xticks(rotation=45)
plt.tight_layout()  # Adjust layout to not cut off labels
plt.show()


In [0]:
# Read the Bronze table as a batch
bronze_df = spark.read.format("delta").load(BRONZE_DELTA)

# Print the count of rows in the Bronze table
print(f"Total number of tweets in the Bronze table: {bronze_df.count()}")


## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# Initialize a Spark session
spark = SparkSession.builder \
    .appName("Silver Data Preprocessing") \
    .getOrCreate()

# Set legacy time parser policy
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [0]:
# Define paths
silver_checkpoint_path = "/tmp/labuser104917-3013268/silver.checkpoint"
silver_delta_path = "/tmp/labuser104917-3013268/silver.delta"
bronze_delta_path = "/tmp/labuser104917-3013268/bronze.delta"

In [0]:
# Load the Bronze Delta table as a streaming DataFrame
bronze_df = spark.readStream.format("delta").load(bronze_delta_path)

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import regexp_replace, col

def clean_bronze(df: DataFrame) -> DataFrame:
    """
    Cleans the text column in the Bronze DataFrame by removing mentions (@username).
    
    Args:
        df (DataFrame): The input DataFrame representing the Bronze layer.

    Returns:
        DataFrame: A DataFrame with a cleaned `cleaned_text` column.
    """
    mention_pattern = r'@\w+'
    clean_df = df.withColumn("cleaned_text", regexp_replace(col("text"), mention_pattern, ""))
    return clean_df


In [0]:
bronze_clean_df = clean_bronze(bronze_df)
silver_df = bronze_clean_df.select(
    col('date').alias('timestamp'),
    col('text').contains('@').cast('integer').alias('mention'),
    col('cleaned_text'),
    col('sentiment'),
)

In [0]:
# Write Silver data to the Delta table as a streaming DataFrame
silver_stream = silver_df.writeStream \
    .format("delta") \
    .option("checkpointLocation", silver_checkpoint_path) \
    .option("path", silver_delta_path) \
    .outputMode("append") \
    .queryName("silver_stream") \
    .start()


In [0]:

# Verify the Silver Delta table
silver_df = spark.read.format("delta").load(silver_delta_path)
row_count = silver_df.count()
print(f"Silver Delta Row count: {row_count}")
silver_df.show(5)


In [0]:
print(silver_stream.status)

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

Setup read stream

In [0]:
# Initialize the Spark session
spark = SparkSession.builder \
    .appName("Transform Silver to Gold Data") \
    .getOrCreate()

# Define paths
silver_delta_path = "/tmp/labuser104917-3013268/silver.delta"

# Load the Silver Delta table as a streaming DataFrame
silver_df = spark.readStream.format("delta").load(silver_delta_path)



setup a write stream

In [0]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, IntegerType, FloatType
import mlflow.pyfunc

# Initialize the Spark session
spark = SparkSession.builder \
    .appName("Transform Silver to Gold Data") \
    .getOrCreate()

# Define paths
silver_checkpoint_path = "/tmp/labuser104917-3013268/silver.checkpoint"
silver_delta_path = "/tmp/labuser104917-3013268/silver.delta"
gold_checkpoint_path = "/tmp/labuser104917-3013268/gold.checkpoint"
gold_delta_path = "/tmp/labuser104917-3013268/gold.delta"

# Load the Silver Delta table as a streaming DataFrame
silver_df = spark.readStream.format("delta").load(silver_delta_path)

# Load the pretrained transformer sentiment classifier from the MLflow registry
model_name = "HF_TWEET_SENTIMENT"
sentiment_model_uri = f"models:/{model_name}/Production"
sentiment_classifier = mlflow.pyfunc.spark_udf(
    spark,
    model_uri=sentiment_model_uri,
    env_manager="conda"
)

# Define UDFs for mapping sentiment to integer IDs and predicted score
def parse_sentiment_output(output):
    sentiment_map = {"POS": "positive", "NEG": "negative", "NEU": "neutral"}
    sentiment, score = output.split(", ")
    sentiment = sentiment_map[sentiment.strip("{}")]
    score = float(score.split(":")[1])
    return sentiment, score

def sentiment_to_id(sentiment):
    return {"positive": 1, "negative": 0, "neutral": 2}.get(sentiment, 2)

# Register UDFs
sentiment_udf = udf(sentiment_to_id, IntegerType())
extract_sentiment = udf(lambda output: parse_sentiment_output(output)[0], StringType())
extract_score = udf(lambda output: parse_sentiment_output(output)[1], FloatType())

# Apply the model to the Silver DataFrame and transform to Gold Data
gold_df = silver_df.withColumn("predicted_output", sentiment_classifier(col("cleaned_text"))) \
    .withColumn("predicted_sentiment", extract_sentiment(col("predicted_output"))) \
    .withColumn("predicted_score", extract_score(col("predicted_output"))) \
    .withColumn("predicted_sentiment_id", sentiment_udf(col("predicted_sentiment"))) \
    .withColumn("sentiment_id", sentiment_udf(col("sentiment")))

# Select relevant columns for the Gold Delta table
gold_df = gold_df.select(
    col("timestamp"),
    col("mention"),
    col("cleaned_text"),
    col("sentiment"),
    col("predicted_score"),
    col("predicted_sentiment"),
    col("sentiment_id"),
    col("predicted_sentiment_id")
).limit(100)

# Write Gold data to the Delta table as a streaming DataFrame
gold_stream = gold_df.writeStream \
    .format("delta") \
    .option("checkpointLocation", gold_checkpoint_path) \
    .option("path", gold_delta_path) \
    .outputMode("append") \
    .queryName("gold_stream") \
    .start()


In [0]:
print(gold_stream.status)

In [0]:
# Load the Gold Delta table in batch mode
gold_df_batch = spark.read.format("delta").load(gold_delta_path)

# Display the first 5 rows
gold_df_batch.show(5)

# Alternatively, count the rows and display a summary
print(f"Total rows in the Gold Delta table: {gold_df_batch.count()}")
gold_df_batch.describe().show()


In [0]:
# Load the Silver Delta table as a streaming DataFrame
silver_df = spark.readStream.format("delta").load(silver_delta_path)

In [0]:

# Load the pretrained transformer sentiment classifier from the MLflow registry
model_name = "HF_TWEET_SENTIMENT"
sentiment_model_uri = f"models:/{model_name}/Production"
sentiment_classifier = mlflow.pyfunc.spark_udf(spark, model_uri=sentiment_model_uri)

In [0]:
def sentiment_to_id(sentiment):
    return {"positive": 1, "negative": 0, "neutral": 2}.get(sentiment, 2)

sentiment_udf = udf(sentiment_to_id, IntegerType())
predicted_score_udf = udf(lambda sentiment: 100 if sentiment == "positive" else 0 if sentiment == "negative" else 50, FloatType())


In [0]:
gold_df = silver_df.withColumn("predicted_sentiment", sentiment_classifier(col("cleaned_text"))) \
    .withColumn("predicted_score", predicted_score_udf(col("predicted_sentiment"))) \
    .withColumn("predicted_sentiment_id", sentiment_udf(col("predicted_sentiment"))) \
    .withColumn("sentiment_id", sentiment_udf(col("sentiment")))


In [0]:
gold_df = gold_df.select(
    col("timestamp"),
    col("mention"),
    col("cleaned_text"),
    col("sentiment"),
    col("predicted_score"),
    col("predicted_sentiment"),
    col("sentiment_id"),
    col("predicted_sentiment_id")
).limit(100)

In [0]:
# Write Gold data to the Delta table as a streaming DataFrame
gold_stream = gold_df.writeStream \
    .format("delta") \
    .option("checkpointLocation", gold_checkpoint_path) \
    .option("path", gold_delta_path) \
    .outputMode("append") \
    .queryName("gold_stream") \
    .start()



In [0]:
print(gold_stream.status)

In [0]:
# Read the Bronze table as a batch
gold_df = spark.read.format("delta").load(gold_delta_path)

# Print the count of rows in the Bronze table
print(f"Total number of tweets in the gold table: {gold_df.count()}")
print(gold_df)

In [0]:
gold_df_batch = spark.read.format("delta").load(gold_delta_path)

# Register the DataFrame as a SQL temporary view
gold_df_batch.createOrReplaceTempView("gold_data")

# Run an SQL query
spark.sql("SELECT * FROM gold_data LIMIT 100").show()

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialize Spark session
spark = SparkSession.builder.appName("ML Metrics").getOrCreate()

# Load the Gold Delta table
gold_df = spark.read.format("delta").load(gold_delta_path)

# Assume 'true_label' is in gold_df and 'predicted_label' as well
# Convert labels if necessary, e.g., from string to index
# For demonstration, assuming labels are already numeric

# Evaluator setup
evaluator = MulticlassClassificationEvaluator(labelCol="true_label", predictionCol="predicted_sentiment")

# Calculate metrics
precision = evaluator.evaluate(gold_df, {evaluator.metricName: "precisionByLabel"})
recall = evaluator.evaluate(gold_df, {evaluator.metricName: "recallByLabel"})
f1_score = evaluator.evaluate(gold_df, {evaluator.metricName: "f1"})

# Log metrics to MLflow
mlflow.log_metric("precision", precision)
mlflow.log_metric("recall", recall)
mlflow.log_metric("f1_score", f1_score)


In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas as pd

# Assume predictions and labels are available in the DataFrame
y_true = gold_df.select("true_label").toPandas()
y_pred = gold_df.select("predicted_sentiment_index").toPandas()

# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d")
plt.title('Confusion Matrix')
plt.ylabel('Actual Labels')
plt.xlabel('Predicted Labels')

# Save the figure
confusion_matrix_path = "confusion_matrix.png"
plt.savefig(confusion_matrix_path)
plt.close()

# Log the confusion matrix as an artifact
mlflow.log_artifact(confusion_matrix_path)


In [0]:
# Log model details
model_name = "HF_TWEET_SENTIMENT"
mlflow.log_param("model_name", model_name)
mlflow.log_param("MLflow_version", mlflow.__version__)

# Assume the version of the Delta Table (input-silver) is known
silver_table_version = "version_here"  # Replace with actual version retrieval
mlflow.log_param("silver_table_version", silver_table_version)


## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
total_mentions = gold_df.select("mention").distinct().count()
print(f"Total distinct mentions: {total_mentions}")


In [0]:
from pyspark.sql.functions import col, count

sentiment_counts = gold_df.groupBy("mention", "sentiment") \
    .agg(count("sentiment").alias("count")) \
    .groupBy("mention") \
    .pivot("sentiment", ["neutral", "positive", "negative"]) \
    .sum("count")


In [0]:
from pyspark.sql.functions import expr

sentiment_counts = sentiment_counts.withColumn("total_mentions", expr("coalesce(neutral, 0) + coalesce(positive, 0) + coalesce(negative, 0)"))


In [0]:
sorted_mentions = sentiment_counts.orderBy(col("total_mentions").desc())


In [0]:
top_positive_mentions = sorted_mentions.orderBy(col("positive").desc()).limit(20).toPandas()

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.bar(top_positive_mentions['mention'], top_positive_mentions['positive'], color='green')
plt.xlabel('Mention')
plt.ylabel('Number of Positive Tweets')
plt.title('Top 20 Positive Mentions')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [0]:
top_negative_mentions = sorted_mentions.orderBy(col("negative").desc()).limit(20).toPandas()

plt.figure(figsize=(10, 8))
plt.bar(top_negative_mentions['mention'], top_negative_mentions['negative'], color='red')
plt.xlabel('Mention')
plt.ylabel('Number of Negative Tweets')
plt.title('Top 20 Negative Mentions')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# ENTER YOUR CODE HERE

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

In [0]:
# Delete the Delta directories
dbutils.fs.rm("dbfs:/delta/silver", recurse=True)



In [0]:
# Delete the Delta tables using PySpark
def delete_delta_tables(*paths):
    for path in paths:
        spark.sql(f"DROP TABLE IF EXISTS {path}")
        dbutils.fs.rm(f"dbfs:/delta/{path}", recurse=True)

# Delete the bronze, silver, and gold tables
delete_delta_tables("gold")


In [0]:
# Drop Delta Tables and Their Directories
spark.sql("DROP TABLE IF EXISTS gold_table")

dbutils.fs.rm("dbfs:/delta/gold", recurse=True)


In [0]:

GOLD_CHECKPOINT = "/tmp/labuser104917-3013268/gold.checkpoint"
GOLD_DELTA = "/tmp/labuser104917-3013268/gold.delta"
# Function to remove directory recursively
def clear_directory(path):
    dbutils.fs.rm(path, recurse=True)
    
clear_directory(GOLD_CHECKPOINT)
clear_directory(GOLD_DELTA)

### ENTER YOUR MARKDOWN HERE